<a href="https://colab.research.google.com/github/BernardoRMendes/Projeto/blob/main/FatorQualityValue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Na literatura de fatores, é comum ordenar de forma sequencial ou criar um portfólio com a interseção dos fatores escolhidos. Um exemplo do tipo sequencial seria escolher primeiro ações do tipo "Value" para então selecionar aquelas que possuem maior Qualidade. Para o segundo caso, podemos pensar em criar um Score separadamente para cada fator. Depois, basta escolher as ações com maior Score combinado.**

**Em geral, existem razões para que ações com baixos múltiplos (consideradas de Valor) estejam baratas, assim como é esperado que ações com maior qualidade negociem a múltiplos mais caros.**

**Nesse estudo, proponho um método de combinar Qualidade e Valor, de modo que possamos comparar o quanto estamos recebendo de Qualidade em relação a cada unidade de Valor. Por exemplo, considere duas ações e seus scores de qualidade e valor, em que a escala de qualidade varia de 0 a 10, com 10 representando maior qualidade, e a escala de valor varia de 0 a 10, com 10 representando mais caro.**

**Ação / Qualidade / Valor / Score**

**A / 9 / 10 / 0,9**

**B / 6 / 5 / 1,2**

**O Score foi calculado simplesmente pela divisão entre Qualidade e Valor. É preferível uma ação de qualidade mediana e valuation justo do que uma ação de qualidade excelente, porém com valuation elevado. Nesse exemplo, é mais vantajoso comprar a ação B em relação à A (Score 1,2 vs 0,9).**

**No estudo abaixo, foram selecionadas 140 ações brasileiras. Como proxy de Qualidade, usaremos o ROIC, enquanto para calcular o Valor, criaremos uma combinação das métricas Preço sobre Receira (PSR) e EV/EBIT. Como forma de deixar o modelo mais robusto, poderíamos combinar o ROIC com outras métricas de qualidade, como Altman Z-Score, Piotroski F-score e cálculo da distância para o default (Modelo de Merton).**

**Os dados foram retirados do site StatusInvest e colocados em uma planilha excel.**

In [1]:
import numpy as np
import pandas as pd

In [2]:
url="https://raw.githubusercontent.com/BernardoRMendes/Projeto/main/qualxvalue.xlsx"

In [3]:
df=pd.read_excel(url,index_col=0)

In [4]:
df.head()

,PRECO,EV/EBIT,PSR,ROIC
TICKER,,,,
ABEV3,15.52,14.44,4.18,17.72
AERI3,8.69,34.49,3.02,9.41
ALPA4,38.31,74.42,6.71,8.16
ALSO3,26.84,17.85,8.93,3.37
ALUP11,26.10,4.40,1.25,14.02


In [5]:
df.isnull().sum()

PRECO      0
EV/EBIT    0
PSR        1
ROIC       0
dtype: int64

In [6]:
df=df.dropna(axis=0)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140 entries, ABEV3 to YDUQ3
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   PRECO    140 non-null    float64
 1   EV/EBIT  140 non-null    float64
 2   PSR      140 non-null    float64
 3   ROIC     140 non-null    float64
dtypes: float64(4)
memory usage: 5.5+ KB


In [8]:
df.describe()

,PRECO,EV/EBIT,PSR,ROIC
count,140.000000,140.000000,140.000000,140.000000
mean,26.963214,17.620786,4.210643,7.866429
std,19.466718,96.697493,5.552319,10.023508
min,1.910000,-822.900000,0.190000,-37.260000
25%,12.975000,6.282500,1.202500,3.770000
50%,22.865000,12.925000,2.410000,8.190000
75%,34.492500,25.660000,4.977500,11.712500
max,103.490000,461.510000,33.350000,45.220000


In [9]:
df.shape

(140, 4)

**A função np.clip() será usada para mitigar o impacto de outliers e evitar distorções na hora de normalizar os dados.**

In [10]:
df["ROIC"]=np.clip(df["ROIC"],df["ROIC"].quantile(0.05),df["ROIC"].quantile(0.95))

In [11]:
df["PSR"]=np.clip(df["PSR"],df["PSR"].quantile(0.05),df["PSR"].quantile(0.95))

In [12]:
df["EV/EBIT"]=np.clip(df["EV/EBIT"],df["EV/EBIT"].quantile(0.05),df["EV/EBIT"].quantile(0.95))

In [13]:
df

,PRECO,EV/EBIT,PSR,ROIC
TICKER,,,,
ABEV3,15.52,14.44,4.18,17.72
AERI3,8.69,34.49,3.02,9.41
ALPA4,38.31,74.42,6.71,8.16
ALSO3,26.84,17.85,8.93,3.37
ALUP11,26.10,4.40,1.25,14.02
...,...,...,...,...
VIVT3,45.05,12.59,1.75,6.45
VIVT4,45.34,12.59,1.78,6.45
VVAR3,12.63,16.13,0.69,10.20


**As métricas de ROIC, EV/EBIT e PSR serão normalizadas para o intervalo entre 10 e 100.**

In [14]:
df["Qualidade"]=0.9*((df["ROIC"]-df["ROIC"].min())/(df["ROIC"].max()-df["ROIC"].min())*100)+10

In [15]:
df["Valuation1"]=0.9*((df["EV/EBIT"]-df["EV/EBIT"].min())/(df["EV/EBIT"].max()-df["EV/EBIT"].min())*100)+10

In [16]:
df["Valuation2"]=0.9*((df["PSR"]-df["PSR"].min())/(df["PSR"].max()-df["PSR"].min())*100)+10

In [17]:
df

,PRECO,EV/EBIT,PSR,ROIC,Qualidade,Valuation1,Valuation2
TICKER,,,,,,,
ABEV3,15.52,14.44,4.18,17.72,83.335438,44.674614,39.274675
AERI3,8.69,34.49,3.02,9.41,59.120975,57.994907,30.433586
ALPA4,38.31,74.42,6.71,8.16,55.478607,84.522551,58.557395
ALSO3,26.84,17.85,8.93,3.37,41.521053,46.940061,75.477410
ALUP11,26.10,4.40,1.25,14.02,72.554028,38.004503,16.943304
...,...,...,...,...,...,...,...
VIVT3,45.05,12.59,1.75,6.45,50.495848,43.445560,20.754118
VIVT4,45.34,12.59,1.78,6.45,50.495848,43.445560,20.982767
VVAR3,12.63,16.13,0.69,10.20,61.422952,45.797372,12.675192


**A composição da métrica de Valuation será a média entre Valuation1 e Valuation2.**

In [18]:
df["Valuation"]=(df["Valuation1"]+df["Valuation2"])/2

In [19]:
df.sort_values(by=['Qualidade'],ascending=False)

,PRECO,EV/EBIT,PSR,ROIC,Qualidade,Valuation1,Valuation2,Valuation
TICKER,,,,,,,,
CMIN3,9.34,8.030,3.79,23.4390,100.0,40.416107,36.302240,38.359173
TASA4,22.95,6.710,1.24,23.4390,100.0,39.539160,16.867087,28.203124
MOSI3,21.67,43.780,11.82,23.4390,100.0,64.166753,97.503917,80.835335
ODPV3,13.32,12.270,4.00,23.4390,100.0,43.232967,37.902782,40.567874
INTB3,21.80,17.670,3.33,23.4390,100.0,46.820477,32.796291,39.808384
...,...,...,...,...,...,...,...,...
BKBR3,10.09,-7.220,1.24,-7.4475,10.0,30.284712,16.867087,23.575900
SOMA3,14.86,-37.753,5.70,-7.4475,10.0,10.000000,50.859550,30.429775
AMAR3,5.43,-7.920,0.66,-7.4475,10.0,29.819665,12.446543,21.133104


In [20]:
df.sort_values(by=['Valuation'],ascending=False)

,PRECO,EV/EBIT,PSR,ROIC,Qualidade,Valuation1,Valuation2,Valuation
TICKER,,,,,,,,
CASH3,33.18,97.717,12.1475,5.56,47.902482,100.000000,100.000000,100.000000
LWSA3,25.85,97.717,12.1475,4.13,43.735613,100.000000,100.000000,100.000000
RDOR3,68.15,87.470,9.6500,4.25,44.085280,93.192367,80.964983,87.078675
LINX3,38.15,97.717,8.3500,0.81,34.061483,100.000000,71.056866,85.528433
SQIA3,22.83,97.717,7.6600,0.99,34.585984,100.000000,65.797942,82.898971
...,...,...,...,...,...,...,...,...
SBFG3,27.03,-37.753,2.7100,-4.32,19.113205,10.000000,28.070881,19.035441
CEAB3,11.81,-20.200,0.8900,-6.56,12.586081,21.661401,14.199517,17.930459
MYPK3,12.52,-29.300,0.3390,-2.09,25.611189,15.615782,10.000000,12.807891


**Cálculo do Score:**

In [21]:
df["Score"]=df["Qualidade"]/df["Valuation"]

**Escolhendo as 20 ações com maior Score e calculando o Peso de cada uma de forma proporcional ao Score.**

In [22]:
df2=df.sort_values(by=['Score'],ascending=False)[:20]

In [23]:
df2["Pesos"]=df2["Score"]/df2["Score"].sum()*100

In [24]:
df2.sort_values(by=['Pesos'],ascending=False)

,PRECO,EV/EBIT,PSR,ROIC,Qualidade,Valuation1,Valuation2,Valuation,Score,Pesos
TICKER,,,,,,,,,,
MRFG3,18.32,3.65,0.339,23.400,99.886358,37.506238,10.000000,23.753119,4.205189,7.453785
TASA4,22.95,6.71,1.240,23.439,100.000000,39.539160,16.867087,28.203124,3.545707,6.284838
BRDT3,22.49,6.88,0.339,17.540,82.810937,39.652100,10.000000,24.826050,3.335647,5.912503
JBSS3,32.29,6.17,0.339,15.440,76.691759,39.180409,10.000000,24.590204,3.118793,5.528125
UNIP6,75.53,8.74,1.850,21.100,93.184401,40.887798,21.516281,31.202039,2.986484,5.293605
BEEF3,11.25,6.29,0.339,13.900,72.204361,39.260131,10.000000,24.630066,2.931554,5.196240
CRFB3,23.09,10.19,0.640,15.000,75.409645,41.851111,12.294110,27.072611,2.785459,4.937284
CYRE3,25.39,4.95,2.640,20.530,91.523481,38.369897,27.537367,32.953632,2.777341,4.922895
TIET11,12.54,4.71,2.490,19.400,88.230780,38.210452,26.394123,32.302288,2.731410,4.841480
